In [1]:
from create_model import create_model
import torch
import warnings
warnings.simplefilter("ignore", category=FutureWarning)
from data_preprocess import input_processor, preprocess, predict_sentence
link_to_in_tokenizer = r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\tokenizer.pickle"
link_to_out_tokenizer = r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\out_tokenizer.pickle"
link_to_in_embedding = r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\in_embedding.npy" #chủ động thay đổi đường dẫn tới file
link_to_out_embedding = r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\out_embedding.npy"#chủ động thay đổi đường dẫn tới file
model = create_model(link_to_in_embedding,link_to_out_embedding) #load model đã train
model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Saved\best_model.pth")) #chủ động thay đổi đường dẫn tới file
input_sentence = "he are the best football player." #câu cần sửa
output_sentence = predict_sentence(model,input_sentence, link_to_in_tokenizer, link_to_out_tokenizer) #dự đoán câu
print("Câu gốc:", input_sentence)
print("Câu đã sửa:", output_sentence)

Câu gốc: he are the best football player.
Câu đã sửa: he is the best football player .


In [ ]:
output = []
test_sentences = []
import pandas as pd
data = pd.read_csv(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Test\lang_8_cleaned.csv")
data = data[:300]
data.rename(columns={0: "error", 1: "correct"}, inplace=True)
data = data[data['error'].str.split().str.len() <= 32]
error_sentences = [str(sentence) for sentence in data["error"]]
ref_sentences = [str(sentence) for sentence in data["correct"]]

In [52]:
outputs_sentences = []
for sentence in error_sentences[:300]:
    decoded = predict_sentence(model, sentence, link_to_in_tokenizer, link_to_out_tokenizer)
    outputs_sentences.append(decoded)

In [53]:
print(len(outputs_sentences))

300


In [54]:
import re
import unicodedata
def decontracted(phrase): #hàm chuyển từ viết tắt thành từ đầy đủ
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"gon na", " going to", phrase)
    phrase = re.sub(r"wan na", " want to", phrase)
    phrase = re.sub(r"gonna", " going to", phrase)
    phrase = re.sub(r"wanna", " want to", phrase)


    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub("\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

def clean_text(t): #hàm bỏ dấu và ký tự unicode

  t = unicodedata.normalize('NFKD', t).encode('ascii', 'ignore').decode('ascii') 
  t = decontracted(t)

  t = re.sub(r'x D', '', t)
  t = re.sub(r': D', '', t)
  t = re.sub(r': P', '', t)

  t = re.sub(r'xD', '', t)
  t = re.sub(r':D', '', t)
  t = re.sub(r':P', '', t)

  if '(' in t and ')' in t: #loại bỏ nội dung trong dấu ngoặc đơn => Hello (world) => Hello
    try:
      t = re.sub(t.split("(")[-1].split(")")[0], '', t)
    except:
      pass
    #t = re.sub("(", '', t)
    #t = re.sub(")", '', t)

  t = re.sub(r'[^A-Za-z;!?.,\-\' ]+', ' ', t) #loại bỏ các emoji, số, ký tự đặc biệt

#chuẩn hóa dấu câu, thêm khoảng trống trước dâu câu. 
  t = re.sub(r'\.+',r' .',t)
  t = re.sub(r'\;+',r' , ',t)
  t = re.sub(r'!+',r' !',t )
  t = re.sub(r'\?+',r' ?',t )
  t = re.sub(r'\-+',r' - ',t )
  t = re.sub(r'\,+',r' , ',t )
  t = re.sub(r'\'+',r" ' ",t)
  t = re.sub(' +', ' ', t)
  t = t.lower()

  return t #trả về văn bản được làm sạch

In [55]:
ref_sentences = [clean_text(sentence) for sentence in ref_sentences]
error_sentences = [clean_text(sentence) for sentence in error_sentences]

In [56]:
i = 11
print(f"Output sentences: {outputs_sentences[i]}")
print(f"Ref sentences: {ref_sentences[i]}")
print(f"Error sentences: {error_sentences[i]}")

Output sentences: well , finally , my mother took me to the customs window , and she gave me a passport to the man . . .
Ref sentences: ifinally my mother took me to the customs window , and gave a passport to the man . . .
Error sentences: well , finally , my mother took me to the customs window , she gave a passport to man . . .


In [57]:
import nltk
from nltk.util import ngrams
from collections import Counter

# Nếu chưa cài đặt bộ tokenizer:
# nltk.download('punkt')

def sentence_gleu(source: str, hypothesis: str, reference: str, max_n: int = 4) -> float:
    source_tokens = nltk.word_tokenize(source)
    hyp_tokens = nltk.word_tokenize(hypothesis)
    ref_tokens = nltk.word_tokenize(reference)
    
    overlap = 0
    total = 0

    for n in range(1, max_n + 1):
        hyp_ngrams = Counter(ngrams(hyp_tokens, n))
        ref_ngrams = Counter(ngrams(ref_tokens, n))
        src_ngrams = Counter(ngrams(source_tokens, n))

        ref_diff = ref_ngrams - src_ngrams
        hyp_diff = hyp_ngrams - src_ngrams

        match = sum((hyp_diff & ref_diff).values())
        total_hyp = sum(hyp_diff.values())

        overlap += match
        total += total_hyp

    if total == 0:
        return 1.0 if sum((ref_ngrams - src_ngrams).values()) == 0 else 0.0
    return overlap / total

def compute_corpus_gleu(sources, predictions, references, max_n=4):
    """
    Tính GLEU trung bình trên toàn bộ tập dữ liệu.
    - sources: list câu gốc (chưa sửa)
    - predictions: list câu do model sửa
    - references: list câu đúng
    """
    assert len(sources) == len(predictions) == len(references), "Danh sách không khớp độ dài"

    scores = [
        sentence_gleu(src, pred, ref, max_n=max_n)
        for src, pred, ref in zip(sources, predictions, references)
    ]
    return sum(scores) / len(scores)


In [61]:
print(f"GLEU Score of the LSTM Model: {compute_corpus_gleu(error_sentences[:300], outputs_sentences[:], ref_sentences[:300])}")

GLEU Score of the LSTM Model: 0.2876030857422058


In [62]:
!pip install bert-score


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
from bert_score import score
def compute_bert_score(predictions, references, lang="en"):
    """
    Tính điểm BERTScore giữa các câu dự đoán và câu tham chiếu.
    
    Args:
        predictions (list): danh sách các câu đầu ra từ mô hình.
        references (list): danh sách các câu đúng thực tế.
        lang (str): ngôn ngữ (mặc định là "en" cho tiếng Anh).
        
    Returns:
        dict: Precision, Recall, F1 trung bình và danh sách chi tiết.
    """
    P, R, F1 = score(predictions, references, lang=lang)
    
    return {
        "precision": round(P.mean().item(), 4),
        "recall": round(R.mean().item(), 4),
        "f1": round(F1.mean().item(), 4),
        "f1_list": [round(f.item(), 4) for f in F1]
    }

In [64]:
bert_result = compute_bert_score(outputs_sentences[:300], ref_sentences[:300])
print("BERTScore F1:", bert_result["f1"])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.9653


In [65]:
#viết câu sai vào file source.txt
with open(r"C:\Users\Admin\Downloads\errant_eva\source_lang_8.txt", "w", encoding="utf-8") as f:
    for sentence in error_sentences[:300]:
        f.write(sentence + "\n")
    

In [66]:
with open(r"C:\Users\Admin\Downloads\errant_eva\reference_lang_8.txt","w",encoding = "utf-8") as f:
    for sentence in ref_sentences[:300]:
        f.write(sentence + "\n")

with open(r"C:\Users\Admin\Downloads\errant_eva\hypothesis_lang_8.txt","w",encoding = "utf-8") as f:
    for sentence in outputs_sentences[:300]:
        f.write(sentence + "\n")